---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def remove_leaky_features(df):
    return drop_cols(df, ['late_fee', 'judgment_amount'])

def remove_non_responsible_rows(df):
    if 'compliance' not in df.columns:
        return df
    
    return df.dropna(axis='index', subset=['compliance'])

def drop_cols(df, del_cols):   
    for col in (set(del_cols) & set(df.columns)):
        df = df.drop([col], axis=1)
    return df

def extract_compliance(df):
    return df['compliance']

def remove_compliance(df):
    return drop_cols(df, ['compliance'])

def remove_output_columns(df):
    return drop_cols(df, ['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail'])

def remove_unused_features(df):
    return drop_cols(df, ['grafitti_status', 'violator_name', 'admin_fee', 'state_fee', 'clean_up_cost', 'inspector_name'])
    
def clean_training_columns(df):
    cleaned = remove_leaky_features(df)
    cleaned = remove_output_columns(cleaned)
    cleaned = remove_unused_features(cleaned)
    return cleaned

def print_column_details(df):
    print("Shape: ", df.shape)
    for column in df.columns:
        print(column, df[column].dtype, df[column].nunique())
        
def onehot_features(train, test):
    features = train.select_dtypes(include=['object']).columns
    for column in features:
        categories = pd.concat([train[column], test[column]]).dropna().unique()
        train[column] = train[column].astype('category', ordered=False, categories=categories)
        test[column] = test[column].astype('category', ordered=False, categories=categories)

    train = pd.get_dummies(train, columns=features)
    test = pd.get_dummies(test, columns=features)
    return train, test
    
def convert_addresses(df):
    addresses = pd.read_csv('addresses.csv', encoding = 'ISO-8859-1')
    latlons = pd.read_csv('latlons.csv', encoding = 'ISO-8859-1')
    addresses = pd.merge(left=addresses, right=latlons, left_on='address', right_on='address')
    average_lat = addresses['lat'].mean()
    average_lon = addresses['lon'].mean()
    
    df_with_lat_lon = pd.merge(left=df, right=addresses, left_on='ticket_id', right_on='ticket_id', how='left')
    df_with_lat_lon = drop_cols(df_with_lat_lon, ['address', 'violation_street_number', 'violation_street_name', 'violation_zip_code'])
    df_with_lat_lon = drop_cols(df_with_lat_lon, ['mailing_address_str_number', 'mailing_address_str_name', 'city', 'state', 'zip_code', 'non_us_str_code', 'country'])
    
    df_with_lat_lon['lat'].fillna(average_lat, inplace=True)
    df_with_lat_lon['lon'].fillna(average_lon, inplace=True)
    return df_with_lat_lon

def fillna_with_mode(df, field):
    df[field].fillna(df[field].mode()[0], inplace=True)
    return df

def convert_datetime(df, field):
    weekday_field = field + '_weekday'
    df[weekday_field] = df[field].dt.weekday   
    fillna_with_mode(df, weekday_field)
    
    hour_field = field + '_hour'
    df[hour_field] = df[field].dt.hour
    fillna_with_mode(df, hour_field)
    
    return df.drop([field], axis=1)

def convert_datetimes(df):
    df = convert_datetime(df, 'hearing_date')
    df = convert_datetime(df, 'ticket_issued_date')
    return df

def remove_ticket_id(df):
    ticket_ids = df['ticket_id']
    return df.drop(['ticket_id'], axis=1), ticket_ids

def remove_object_columns(df):
    return df.select_dtypes(exclude=['object'])

def train_tree_classifier(X, y):
    print('Training RF classifier')
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    
    clf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
    
    param_grid = {
        'oob_score': [False],
        'max_features': ['sqrt'],
        'min_samples_leaf': [10]
    }

    CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc')
    CV_rfc.fit(X, y)
    
    print('Best params')
    print(CV_rfc.best_params_)
    
    print('Trained RF classifier')
    return CV_rfc

def print_feature_importance(df, clf):
    print("Feature importance")
    for name, importance in zip(df.columns, clf.best_estimator_.feature_importances_):
        print(name, "=", importance)

def predict_and_score(clf, X, y, name):
    y_score = clf.predict_proba(X)[:,1]
    auc = roc_auc_score(y, y_score)
    print(name, 'AUC:', auc)

def transform_blight_data(blight_train):
    blight_train = remove_non_responsible_rows(blight_train)
    
    blight_train = clean_training_columns(blight_train)
    
    blight_train = convert_addresses(blight_train)
    
    blight_train = convert_datetimes(blight_train)
    
    return blight_train

def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    print('Split data into train and test')
    print('Training set size: ', X_train.shape)
    print('Training y size: ', y_train.shape)
    print('Test set size: ', X_test.shape)
    print('Test y size: ', y_test.shape)

    clf = train_tree_classifier(X_train, y_train)
    
    #print_feature_importance(X_train, clf)
        
    predict_and_score(clf, X_train, y_train, 'Train')
    predict_and_score(clf, X_test, y_test, 'Test')
    return clf

def read_blight_csv(file_name):
    return pd.read_csv(file_name, encoding = 'ISO-8859-1', parse_dates=['ticket_issued_date', 'hearing_date'])
    
def blight_model():
    
    blight_train = read_blight_csv('train.csv')
    blight_train = transform_blight_data(blight_train)
    blight_train_y = extract_compliance(blight_train)
    blight_train = remove_compliance(blight_train)    
    blight_train, _ = remove_ticket_id(blight_train)
    
    blight_test = read_blight_csv('test.csv')
    blight_test = transform_blight_data(blight_test)
    blight_test = remove_compliance(blight_test)
    blight_test, blight_test_ticket_ids = remove_ticket_id(blight_test)
    
    blight_train, blight_test = onehot_features(blight_train, blight_test)
    
    model = train_model(blight_train, blight_train_y)
     
    blight_test_predictions = model.predict_proba(blight_test)[:,1]

    return pd.Series(index=blight_test_ticket_ids, data=blight_test_predictions, name='compliance')

blight_model()
